In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np

## preparing the data using PyTorch
    1. extract-get the data from Fashion MNIST IMAGE DATA 
    2. Transform put the data in a tensor 
    3. Load- put the data into an object to make it accessible
